In [6]:
import numpy as np
import scipy as sp
import suite2p as s2p
import dill

import TwoPUtils as tpu
import STX3KO_analyses as stx
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = 'mCherry7'

file_list = stx.ymaze_sess_deets.CTRL_sessions[mouse]
for f in file_list[6:]:
    print(f)


({'date': '20_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': -1, 'ravel_ind': 6}, {'date': '20_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 5, 'novel_arm': -1, 'ravel_ind': 7})
{'date': '21_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4, 'novel_arm': -1, 'ravel_ind': 8}
({'date': '22_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3, 'novel_arm': -1, 'ravel_ind': 9}, {'date': '22_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 6, 'novel_arm': -1, 'ravel_ind': 10})


/home/mplitt/anaconda3/envs/STX3_test/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
def check_and_make_h5s(file_list, runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        source_folder  = os.path.join('/mnt','BigDisk','2P_scratch')
        source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
        info = tpu.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ =  tpu.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=False)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [4]:
def run_session(fdict):
    _h5list,_frs = check_and_make_h5s([fdict,],runh5=True)
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')

    ops = tpu.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 


for f in file_list[7:]: #file_list:
    
    if isinstance(f,tuple):
        for _f in f:
            run_session(_f)
    else:
        run_session(f)
    
    

{'date': '21_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4, 'novel_arm': -1, 'ravel_ind': 8}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 38627
['/mnt/BigDisk/2P_scratch/mCherry7/21_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_004']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/mCherry7/21_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_004/YMaze_RewardReversal_001_004.h5']
time 1546.67 sec. Wrote 38627 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not regi

In [4]:
def update_sess_dict(mouse, f):
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])

            
    source_folder = "/mnt/BigDisk/2P_scratch"
    source_stem = os.path.join(source_folder, mouse, f['date'], f['scene'], scan_str)

    f['mouse']=mouse
    f.update({'scan_file': source_stem + '.sbx',
              'scanheader_file': source_stem + '.mat',
              'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
              'scan_number': f['scan'],
              'prompt_for_keys': False,
              'VR_only': False,
              'scanner': "NLW",
             })
    return f

def run_and_save(fdict,comb_frames=0):
#     with open(os.path.join('/home/mplitt/YMazeSessPkls',mouse,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
#         sess = dill.load(file)
#     sess.novel_arm = fdict['novel']    
            
    sess = stx.session.YMazeSession(**fdict)
    sess.load_scan_info()
    sess.align_VR_to_2P()
    sess.load_suite2p_data(which_ts=('F', 'Fneu','spks'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
    sess.get_trial_info()
    sess.neuropil_corrected_dff()
    
    
    sess.place_cells_calc(nperms=1000)
    
    sess.add_pos_binned_trial_matrix('F_dff')
    sess.add_timeseries(F_dff_norm = sess.timeseries['F_dff']/np.nanpercentile(sess.timeseries['F_dff'],99,axis=1,keepdims=True))
    sess.rm_pos_binned_trial_matrix('F_dff_norm')
    sess.add_pos_binned_trial_matrix('F_dff_norm')
    
    sess.add_pos_binned_trial_matrix('spks') #,speed=sess.vr_data['dz']._values, speed_thr = .05)  
    sess.add_timeseries(spks_norm = sess.timeseries['spks']/np.nanpercentile(sess.timeseries['spks'],99,axis=1,keepdims=True))
    sess.add_pos_binned_trial_matrix('spks_norm') 

    sess.add_timeseries(licks=sess.vr_data['lick']._values)
    sess.add_pos_binned_trial_matrix('licks')
    sess.add_timeseries(speed=sess.vr_data['dz']._values)
    sess.add_pos_binned_trial_matrix('speed')
    
    tpu.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')
    return comb_frames+sess.scan_info['max_idx']

In [10]:
for f in file_list[6:]: #file_list:
    print(f)
    if isinstance(f,tuple):
        for _f in f:
            f_ = update_sess_dict(mouse,_f)
            _ = run_and_save(f_)
    else:
        f_ = update_sess_dict(mouse,f) 
        _ = run_and_save(f_)

({'date': '20_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': -1, 'ravel_ind': 6, 'mouse': 'mCherry7', 'scan_file': '/mnt/BigDisk/2P_scratch/mCherry7/20_10_2021/YMaze_LNovel/YMaze_LNovel_001_003.sbx', 'scanheader_file': '/mnt/BigDisk/2P_scratch/mCherry7/20_10_2021/YMaze_LNovel/YMaze_LNovel_001_003.mat', 'vr_filename': '/home/mplitt/VR_scratch/mCherry7/20_10_2021/YMaze_LNovel_1.sqlite', 'scan_number': 3, 'prompt_for_keys': False, 'VR_only': False, 'scanner': 'NLW'}, {'date': '20_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 5, 'novel_arm': -1, 'ravel_ind': 7, 'mouse': 'mCherry7', 'scan_file': '/mnt/BigDisk/2P_scratch/mCherry7/20_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_005.sbx', 'scanheader_file': '/mnt/BigDisk/2P_scratch/mCherry7/20_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_005.mat', 'vr_filename': '/home/mplitt/VR_scratch/mCherry7/20_10_2021/YMaze_RewardReversal_1.sqlite', 'scan_number': 5, 'prompt_for_keys': False, 

/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


/mnt/BigDisk/2P_scratch/mCherry7/20_10_2021/YMaze_LNovel/YMaze_LNovel_001_003/suite2p
198 3920
4854 8515
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
Fixing teleports
(128681,) (26827,)
1729.973677581864 1729.8377833753148
/mnt/BigDisk/2P_scratch/mCherry7/20_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_005/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


166 4820
5757 10700
11644 16419
17356 26826
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '21_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4, 'novel_arm': -1, 'ravel_ind': 8}
Fixing teleports
(175003,) (38627,)
2490.822670025189 2490.7445843828714
/mnt/BigDisk/2P_scratch/mCherry7/21_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_004/suite2p
176 5119
6056 10847
11788 16499
17437 22354
23294 28288
29234 38626
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
({'date': '22_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3, 'novel_arm': -1, 'ravel_ind': 9, 'mouse': 'mCherry7', 'scan_file': '/mnt/BigDisk/2P_scratch/mCherry7/22_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_003.sbx

/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


183 4860
5807 11177
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
Fixing teleports
(131780,) (26932,)
1736.699370277078 1736.608564231738
/mnt/BigDisk/2P_scratch/mCherry7/22_10_2021/YMaze_LNovel/YMaze_LNovel_001_006/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


314 4908
6086 10706
11640 16409
17357 26931
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


In [4]:
def single_mouse_aligner(mouse, sessions):
    sess_dir = os.path.join('/home/mplitt/YMazeSessPkls',mouse)
    sess_list = []
    for sess_deets in sessions:     
        if isinstance(sess_deets,(list,tuple)):
            for _sess_deets in sess_deets:
                with open(os.path.join(sess_dir,_sess_deets['date'],"%s_%d.pkl" % (_sess_deets['scene'], _sess_deets['session'])),'rb') as file:
                    sess_list.append(dill.load(file))
        else:
            with open(os.path.join(sess_dir,sess_deets['date'],"%s_%d.pkl" % (sess_deets['scene'], sess_deets['session'])),'rb') as file:
                    sess_list.append(dill.load(file))

    sa = tpu.roi_matching.ROIAligner(sess_list)
    sa.run_pairwise_matches()

    with open(os.path.join(sess_dir,'roi_aligner_results.pkl'), 'wb') as file:
        dill.dump(sa.match_inds,file)

In [5]:
mouse = 'mCherry7'
sessions = stx.ymaze_sess_deets.CTRL_sessions[mouse]
single_mouse_aligner(mouse,sessions)

thresh 0.34936345911026
thresh 0.3356823453903198
thresh 0.3309878239631653
thresh 0.34783749079704285
thresh 0.3291102478504181
thresh 0.301371378660202
thresh 0.33539502120018005
thresh 0.3660072515010834
thresh 0.3054354319572449
thresh 0.3570229539871216
thresh 0.3316214213371277
thresh 0.28150696849823
thresh 0.2960200140476227
thresh 0.34499279952049255
thresh 0.3090883026123047
thresh 0.3375856111049652
thresh 0.3160053322315216
thresh 0.3552974591255188
thresh 0.3298457989692688
thresh 0.32701967453956604
thresh 0.3160980472564697
thresh 0.3383180031776428
thresh 0.31512360072135925
thresh 0.3890476653575897
thresh 0.28878648376464844
thresh 0.32233251428604126
thresh 0.3329576382637024
thresh 0.32873855328559876
thresh 0.3245868215560913
thresh 0.341002179145813
thresh 0.3375217447280884
thresh 0.30560774898529053
thresh 0.290434015750885
thresh 0.3699323663711548
thresh 0.351617915391922
thresh 0.3301163146495819
thresh 0.32321102714538574
thresh 0.3589162061214447
thresh 0.3